In [ ]:
import os
import sys
import csv
import pandas as pd

import ccxt  # noqa: E402

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   num_retries = 0
   try:
       num_retries += 1
       ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
       # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
       return ohlcv
   except Exception:
       if num_retries > max_retries:
           raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
   timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
   timedelta = limit * timeframe_duration_in_ms
   now = exchange.milliseconds()
   all_ohlcv = []
   fetch_since = since
   while fetch_since < now:
       ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
       fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)
       all_ohlcv = all_ohlcv + ohlcv
       #if len(all_ohlcv):
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
       #else:
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(fetch_since))
   return exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)


def scrape_candles_to_csv(exchange_id, max_retries, symbol, timeframe, since, limit):
   # instantiate the exchange by id
   exchange = getattr(ccxt, exchange_id)()
   # convert since from string to milliseconds integer if needed
   if isinstance(since, str):
       since = exchange.parse8601(since)
   # preload all markets from the exchange
   exchange.load_markets()
   # fetch all candles
   ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
   # save them to csv file
   #write_to_csv(filename, ohlcv)
   df = pd.DataFrame(ohlcv, dtype=float) 
   df.rename(columns={0: 'date', 1: 'open', 2: 'high',
                   3: 'low', 4: 'close', 5: 'volume'}, inplace=True)
   df['date'] = pd.to_datetime(df['date'], unit='ms')
   df['vwap'] = (df['open'] + 2 * df['high'] + 2*df['low'] + df['close'])/6
   save_name = symbol.split('/')[0]
   df['symbol'] = save_name
   path = 'crypto_data_1min'
   csv_name = path + '/' + save_name + '.csv'
   df.to_csv(csv_name)
   #print('Saved', len(ohlcv), 'candles f